In [1]:
print("Hello")

Hello


In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math

# cap = cv2.VideoCapture(0) # for webcam
# cap.set(3, 640)
# cap.set(4, 480)
cap = cv2.VideoCapture("../road_detection_model/Images/video1.webm") # for video

model = YOLO("../road_detection_model/Models/yolov8n.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", "bed", "dining table",  "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

print(cv2.cuda.getCudaEnabledDeviceCount())
while True:
    success, img = cap.read()
    results = model(img, stream = True)
    for r in results:
        boxes = r.boxes
        for box in boxes:

            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            # x1, y1, w, h = box.xywh[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            bbox = int(x1), int(y1), int(w), int(h)
            # print(x1, y1, x2, y2)
            # cv2.rectangle(img, (x1, y1), (x2, y2), (255, 200 ,255), 3)
            cvzone.cornerRect(img, bbox)

            # confidence
            conf = math.floor(box.conf[0]* 100)/100

            # class name
            cls = box.cls[0]
            cvzone.putTextRect(img, f"{classNames[int(cls)]} {conf}", (max(0, x1), max(35, y1)))

    cv2.imshow("Image", img)
    cv2.waitKey(1)


0

0: 384x640 5 cars, 1 bus, 4 trucks, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 bus, 2 trucks, 9.1ms
Speed: 2.1ms preprocess, 9.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 5 trucks, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 bus, 4 trucks, 9.1ms
Speed: 2.1ms preprocess, 9.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 4 trucks, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 9.0ms
Speed: 2.1ms preprocess, 9.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 3 trucks, 8.3ms
Speed: 3.2ms preprocess, 8.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 4 trucks, 8.4ms
Spe


0: 384x640 4 cars, 3 trucks, 8.4ms
Speed: 1.9ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 3 trucks, 8.5ms
Speed: 2.5ms preprocess, 8.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 bus, 3 trucks, 7.2ms
Speed: 2.0ms preprocess, 7.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 3 trucks, 7.2ms
Speed: 2.0ms preprocess, 7.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 3 trucks, 7.6ms
Speed: 2.4ms preprocess, 7.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 4 trucks, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 3 trucks, 8.7ms
Speed: 1.8ms preprocess, 8.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 4 trucks, 7.2ms
Speed: 2.1ms pre

Speed: 2.1ms preprocess, 7.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 trucks, 1 fire hydrant, 7.2ms
Speed: 2.2ms preprocess, 7.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 3 trucks, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9 cars, 1 bus, 2 trucks, 8.3ms
Speed: 1.9ms preprocess, 8.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 bus, 3 trucks, 8.1ms
Speed: 1.9ms preprocess, 8.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 3 trucks, 7.4ms
Speed: 2.3ms preprocess, 7.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 2 buss, 3 trucks, 7.2ms
Speed: 2.1ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 car


0: 384x640 1 person, 11 cars, 2 trucks, 7.4ms
Speed: 2.2ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 2 trucks, 7.2ms
Speed: 2.1ms preprocess, 7.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 2 trucks, 7.7ms
Speed: 2.1ms preprocess, 7.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 truck, 7.7ms
Speed: 2.4ms preprocess, 7.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 2 trucks, 7.8ms
Speed: 2.0ms preprocess, 7.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 2 trucks, 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 2 trucks, 7.6ms
Speed: 1.9ms preprocess, 7.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 38